# Version 2

In [ ]:
import numpy as np
import pandas as pd
from itertools import product
from scipy.stats import poisson, norm
from scipy.optimize import minimize_scalar

# Define parameter ranges
season_lengths = [30, 45, 60, 75, 90]
sale_prices = [10, 20, 30, 40, 50]
cost_percentages = [0.15, 0.3, 0.45, 0.6, 0.75]
salvage_percentages = [0.15, 0.3, 0.45, 0.6, 0.75]

In [ ]:
# User inputs
lambda_value = float(input("Enter the average daily customer demand (lambda value): "))
num_simulations = int(input("Enter the number of simulations to run: "))
thief_percentages = np.arange(0, 0.11, 0.01)  # 0% to 10%

Enter the average daily customer demand (lambda value): 40
Enter the number of simulations to run: 1000


In [ ]:

# Generate customer visits and thief visits per season length
people_visits_dict = {
    season_length: np.random.poisson(lambda_value, (num_simulations, season_length))
    for season_length in season_lengths
}
people_visits_dict

{30: array([[50, 47, 35, ..., 34, 45, 37],
        [51, 33, 35, ..., 38, 52, 42],
        [38, 38, 35, ..., 36, 26, 41],
        ...,
        [38, 36, 37, ..., 28, 49, 35],
        [35, 40, 38, ..., 27, 47, 42],
        [35, 44, 38, ..., 31, 50, 49]]),
 45: array([[35, 38, 46, ..., 43, 47, 42],
        [46, 36, 48, ..., 35, 26, 35],
        [38, 35, 36, ..., 44, 39, 30],
        ...,
        [33, 43, 47, ..., 27, 30, 33],
        [39, 40, 35, ..., 52, 31, 36],
        [47, 42, 42, ..., 31, 42, 42]]),
 60: array([[48, 50, 41, ..., 33, 43, 45],
        [43, 48, 36, ..., 39, 34, 45],
        [32, 32, 43, ..., 35, 43, 31],
        ...,
        [40, 30, 46, ..., 38, 34, 38],
        [31, 42, 43, ..., 34, 32, 45],
        [38, 50, 37, ..., 38, 47, 47]]),
 75: array([[44, 37, 41, ..., 32, 42, 47],
        [49, 47, 34, ..., 48, 49, 39],
        [45, 38, 34, ..., 44, 34, 53],
        ...,
        [28, 33, 39, ..., 30, 50, 36],
        [36, 34, 53, ..., 44, 36, 39],
        [33, 42, 42, ..., 37,

In [ ]:
# Generate thief visits per season length and thief percentage outside the main loop
thief_visits_dict = {}

# Generate thief visits for each season length and thief percentage
for season_length in season_lengths:
    thief_visits_dict[season_length] = {
        thief_percentage: np.random.poisson(lambda_value * thief_percentage, (num_simulations, season_length))
        for thief_percentage in thief_percentages
    }
thief_visits_dict

{30: {np.float64(0.0): array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]),
  np.float64(0.01): array([[1, 0, 0, ..., 1, 1, 1],
         [0, 0, 0, ..., 0, 0, 0],
         [1, 0, 1, ..., 1, 1, 1],
         ...,
         [0, 0, 1, ..., 1, 0, 1],
         [0, 0, 0, ..., 0, 1, 0],
         [0, 0, 0, ..., 1, 1, 1]]),
  np.float64(0.02): array([[1, 0, 0, ..., 0, 0, 0],
         [2, 0, 0, ..., 0, 0, 1],
         [0, 1, 0, ..., 0, 1, 2],
         ...,
         [0, 1, 1, ..., 1, 0, 0],
         [1, 3, 0, ..., 1, 1, 2],
         [4, 2, 3, ..., 3, 2, 0]]),
  np.float64(0.03): array([[1, 0, 1, ..., 0, 1, 2],
         [1, 0, 0, ..., 3, 0, 0],
         [0, 2, 1, ..., 4, 3, 0],
         ...,
         [2, 3, 0, ..., 0, 1, 1],
         [1, 0, 1, ..., 1, 1, 0],
         [1, 1, 3, ..., 0, 0, 2]]),
  np.float64(0.04): array([[0, 3, 3, ..., 1, 

In [ ]:
# --- Simulation Function ---
def run_simulation_vectorized(begin_inventory, thief_percentage, num_days, sale_price, unit_cost, salvage_value, people_visits, thief_visits):
    customer_demand = people_visits
    theft_counts = thief_visits

    inventory = np.full((num_simulations, 1), begin_inventory)
    total_sales = np.zeros(num_simulations)

    for day in range(num_days):
        daily_demand = customer_demand[:, day]
        daily_theft = theft_counts[:, day]
        total_demand = daily_demand + daily_theft

        # Use np.where to handle the comparison element-wise
        mask = total_demand > inventory[:, -1]  # Demand exceeds inventory
        theft_for_the_day = np.where(mask, np.ceil((daily_theft / (daily_theft + daily_demand)) * inventory[:, -1]).astype(int), daily_theft)
        customer_sales = np.where(mask, np.minimum(daily_demand, inventory[:, -1] - theft_for_the_day), np.minimum(daily_demand, inventory[:, -1]))

        total_sales += customer_sales
        remaining_inventory = inventory[:, -1] - (customer_sales + theft_for_the_day)
        inventory = np.hstack([inventory, remaining_inventory[:, None]])

    final_inventory = inventory[:, -1]
    total_revenue = (
        total_sales * sale_price -
        begin_inventory * unit_cost +
        final_inventory * salvage_value
    )

    mean_customer_demand = np.mean(customer_demand.sum(axis=1))
    mean_theft = np.mean(theft_counts.sum(axis=1))
    std_customer_demand = np.std(customer_demand.sum(axis=1))
    std_theft = np.std(theft_counts.sum(axis=1))

    return -np.mean(total_revenue), mean_customer_demand, mean_theft, std_customer_demand, std_theft


In [ ]:
# --- Main Loop ---
optimal_results = []

for season_length in season_lengths:
    people_visits = people_visits_dict[season_length]

    # Loop through each thief percentage
    for thief_percentage in thief_percentages:
        thief_visits = thief_visits_dict[season_length][thief_percentage]

        for sale_price, cost_percent, salvage_percent in product(sale_prices, cost_percentages, salvage_percentages):
            num_days = season_length
            unit_cost = sale_price * cost_percent
            salvage_value = unit_cost * salvage_percent

            max_inventory = int(poisson.ppf(0.9999, lambda_value * num_days)) + 500
            min_inventory = int(norm.ppf(0.0001, loc=lambda_value * num_days, scale=np.sqrt(lambda_value * num_days)))

            result = minimize_scalar(
                lambda x: run_simulation_vectorized(x, thief_percentage, num_days, sale_price, unit_cost, salvage_value, people_visits, thief_visits)[0],
                bounds=(min_inventory, max_inventory),
                method='bounded'
            )

            best_order_qty = int(result.x)
            best_revenue, mean_customer_demand, mean_theft, std_customer_demand, std_theft = run_simulation_vectorized(
                best_order_qty, thief_percentage, num_days, sale_price, unit_cost, salvage_value, people_visits, thief_visits
            )


            optimal_results.append([
                season_length, sale_price, cost_percent, salvage_percent, thief_percentage, best_order_qty, -best_revenue,
                mean_customer_demand, mean_theft, std_customer_demand, std_theft
            ])

# Save results
optimal_results_df = pd.DataFrame(
    optimal_results,
    columns=[
        "Season Length", "Sale Price", "Cost Percent", "Salvage Percent", "Theft Percent",
        "Optimal Order Qty", "Max Revenue (Simulated)", "μ (Customer)", "μT (Theft)",
        "σ (Customer)", "σ (Theft)"
    ]
)

output_filename = "NVMS.xlsx"
optimal_results_df.to_excel(output_filename, index=False)

print(f"Results saved to {output_filename}")

Results saved to NVMS.xlsx


Customer & Thief List


In [ ]:
import numpy as np
import pandas as pd
from itertools import product
from scipy.stats import poisson, norm
from scipy.optimize import minimize_scalar

# Define parameter ranges
season_lengths = [30, 45, 60, 75, 90]
sale_prices = [10, 20, 30, 40, 50]
cost_percentages = [0.15, 0.3, 0.45, 0.6, 0.75]
salvage_percentages = [0.15, 0.3, 0.45, 0.6, 0.75]

# User inputs
lambda_value = float(input("Enter the average daily customer demand (lambda value): "))
num_simulations = int(input("Enter the number of simulations to run: "))
thief_percentages = np.arange(0, 0.11, 0.01)  # 0% to 10%

# Generate customer and thief visits
people_visits_dict = {
    season_length: np.random.poisson(lambda_value, (num_simulations, season_length))
    for season_length in season_lengths
}
thief_visits_dict = {
    season_length: {
        thief_pct: np.random.poisson(lambda_value * thief_pct, (num_simulations, season_length))
        for thief_pct in thief_percentages
    }
    for season_length in season_lengths
}

# Simulation: returns revenue stats and stolen totals array
def run_simulation(begin_inventory, people_visits, thief_visits, sale_price, unit_cost, salvage_value):
    inventory = np.full((num_simulations, 1), begin_inventory)
    total_sales = np.zeros(num_simulations)
    total_stolen = np.zeros(num_simulations)

    for day in range(people_visits.shape[1]):
        demand = people_visits[:, day]
        theft = thief_visits[:, day]
        total_demand = demand + theft

        mask = total_demand > inventory[:, -1]
        stolen_today = np.where(
            mask,
            np.ceil((theft / total_demand) * inventory[:, -1]).astype(int),
            theft
        )
        sales_today = np.where(
            mask,
            np.minimum(demand, inventory[:, -1] - stolen_today),
            np.minimum(demand, inventory[:, -1])
        )

        total_sales += sales_today
        total_stolen += stolen_today
        inventory = np.hstack([inventory, (inventory[:, -1] - sales_today - stolen_today)[:, None]])

    final_inventory = inventory[:, -1]
    revenue = total_sales * sale_price - begin_inventory * unit_cost + final_inventory * salvage_value
    return revenue, total_stolen

# Main optimization loop
data_rows = []
stolen_detail_rows = []

for season_length in season_lengths:
    people_visits = people_visits_dict[season_length]
    for thief_pct in thief_percentages:
        thief_visits = thief_visits_dict[season_length][thief_pct]
        for sale_price, cost_pct, salvage_pct in product(sale_prices, cost_percentages, salvage_percentages):
            unit_cost = sale_price * cost_pct
            salvage_value = unit_cost * salvage_pct
            days = season_length

            max_inv = int(poisson.ppf(0.9999, lambda_value * days)) + 500
            min_inv = int(norm.ppf(0.0001, loc=lambda_value * days, scale=np.sqrt(lambda_value * days)))

            res = minimize_scalar(
                lambda Q: -np.mean(run_simulation(int(Q), people_visits, thief_visits, sale_price, unit_cost, salvage_value)[0]),
                bounds=(min_inv, max_inv), method='bounded'
            )
            best_Q = int(res.x)

            # Get revenue and stolen details for best Q
            rev_array, stolen_array = run_simulation(best_Q, people_visits, thief_visits, sale_price, unit_cost, salvage_value)
            mean_rev = np.mean(rev_array)
            mean_cust = np.mean(people_visits.sum(axis=1))
            std_cust = np.std(people_visits.sum(axis=1))
            mean_planned = np.mean(thief_visits.sum(axis=1))
            std_planned = np.std(thief_visits.sum(axis=1))
            mean_stolen = np.mean(stolen_array)
            std_stolen = np.std(stolen_array)

            # Summary row
            data_rows.append([
                season_length, sale_price, cost_pct, salvage_pct, thief_pct,
                best_Q, mean_rev,
                mean_cust, std_cust,
                mean_planned, std_planned,
                mean_stolen, std_stolen
            ])

            # Detail rows per simulation
            for sim_run, stolen_val in enumerate(stolen_array, start=1):
                stolen_detail_rows.append([
                    season_length, sale_price, cost_pct, salvage_pct, thief_pct,
                    best_Q, sim_run, stolen_val
                ])

# Create DataFrames
summary_df = pd.DataFrame(data_rows, columns=[
    "Season Length", "Sale Price", "Cost Percent", "Salvage Percent", "Theft Percent",
    "Optimal Q", "Mean Revenue",
    "μ Customer", "σ Customer",
    "μ Planned Theft", "σ Planned Theft",
    "μ Stolen", "σ Stolen"
])

details_df = pd.DataFrame(stolen_detail_rows, columns=[
    "Season Length", "Sale Price", "Cost Percent", "Salvage Percent", "Theft Percent",
    "Optimal Q", "Simulation Run", "Stolen Units"
])

# Save summary
summary_df.to_excel("NVMS_summary.xlsx", index=False)

# Save details (split into sheets if too large)
max_rows = 1_000_000
with pd.ExcelWriter("NVMS_stolen_details.xlsx", engine="xlsxwriter") as writer:
    total_rows = len(details_df)
    if total_rows <= max_rows:
        details_df.to_excel(writer, sheet_name="Details", index=False)
    else:
        for i in range((total_rows // max_rows) + 1):
            start = i * max_rows
            end = start + max_rows
            chunk = details_df.iloc[start:end]
            sheet = f"Details_{i+1}"
            chunk.to_excel(writer, sheet_name=sheet, index=False)

print("Files saved: NVMS_summary.xlsx and NVMS_stolen_details.xlsx")


Enter the average daily customer demand (lambda value): 40
Enter the number of simulations to run: 1000


ModuleNotFoundError: No module named 'xlsxwriter'

In [ ]:
!pip install xlsxwriter
# Save details (split into sheets if too large)
max_rows = 1_000_000
with pd.ExcelWriter("NVMS_stolen_details.xlsx", engine="xlsxwriter") as writer:
    total_rows = len(details_df)
    if total_rows <= max_rows:
        details_df.to_excel(writer, sheet_name="Details", index=False)
    else:
        for i in range((total_rows // max_rows) + 1):
            start = i * max_rows
            end = start + max_rows
            chunk = details_df.iloc[start:end]
            sheet = f"Details_{i+1}"
            chunk.to_excel(writer, sheet_name=sheet, index=False)

print("Files saved: NVMS_summary.xlsx and NVMS_stolen_details.xlsx")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 3.3 MB/s eta 0:00:00
Files saved: NVMS_summary.xlsx and NVMS_stolen_details.xlsx
